In [1]:
import math
import numpy as np
import numpy.ma as ma
import pandas as pd
import csv
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from time import strftime, localtime
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import gensim.corpora as corpora
from pprint import pprint
from wordcloud import WordCloud
import os
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pickle 
from gensim.models.ldamulticore import LdaMulticore
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


input_file = r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\twitter data\Tweets about the Top Companies from 2015 to 2020\Tweet.csv\Tweet.csv"
input_df  = pd.read_csv(input_file, index_col="tweet_id")

#TCT = pd.read_csv(r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\twitter data\Tweets about the Top Companies from 2015 to 2020\Company_Tweet.csv\Company_Tweet.csv", index_col="tweet_id")
#TT  = pd.read_csv(r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\twitter data\Tweets about the Top Companies from 2015 to 2020\Tweet.csv\Tweet.csv", index_col="tweet_id")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fabio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Main

def import_twitter_data_period(target_file, filters_dict, period_start, period_end, relavance_lifetime):
    #prep data
    input_df = pd.read_csv(target_file)
    epoch_time  = datetime(1970, 1, 1)
    period_start -= timedelta(seconds=relavance_lifetime)
    epoch_start = (period_start - epoch_time).total_seconds()
    epoch_end   = (period_end - epoch_time).total_seconds()
    if filters_dict != None and filters_dict != dict():
        raise ValueError("filters_dict input not programmed yet")
    
    #trim according to time window    
    input_df = input_df[input_df["post_date"]>epoch_start]
    input_df = input_df[input_df["post_date"]<epoch_end]
    
    return input_df


def update_shortened_company_file(df_stocks_list_file, corp_stopwords, file_location=None):
    stocks_list         = list(df_stocks_list_file["Name"].map(lambda x: x.lower()).values)
            
    stocks_list_shortened_dict = dict()
    for stock_name in stocks_list:
        shortened = False
        stock_name_split = stock_name.split(" ")
        for word in reversed(stock_name_split):
            for stop in corp_stopwords:
                if stop == word:
                    stock_name_split.remove(word)
                    shortened = True
        if shortened == True:
            stocks_list_shortened_dict[stock_name] = " ".join(stock_name_split)
    
    return stocks_list_shortened_dict


def prep_twitter_text_for_subject_discovery(input_list, df_stocks_list_file=None):
    #prep parameters
    death_characters    = ["$", "amazon", "apple", "goog", "tesla", "http", "@", "1", "2", "3", "4", "5", "6", "7", "8", "9", "0", ".", "'s", "compile", "www"]
    stocks_list         = list(df_stocks_list_file["Name"].map(lambda x: x.lower()).values)
    tickers_list        = list(df_stocks_list_file["Ticker"].map(lambda x: x.lower()).values)
    stopwords_english   = stopwords.words('english')
    #these are words are removed from company names to create additional shortened versions of those names. This is so these version can be eliminated from the tweets to make the subjects agnostic
    corp_stopwords      = [".com", "company", "corp", "froup", "fund", "gmbh", "global", "incorporated", "inc.", "inc", "tech", "technology", "technologies", "trust", "limited", "lmt", "ltd"]
    #these are words are directly removed from tweets
    misc_stopwords      = ["iphone", "airpods", "jeff", "bezos", "#microsoft", "#amzn", "volkswagen", "microsoft", "amazon's", "tsla", "androidwear", "ipad", "amzn", "iphone", "tesla", "TSLA", "elon", "musk", "baird", "robert", "pm", "androidwear", "android", "robert", "ab", "ae", "dlvrit", "https", "iphone", "inc", "new", "dlvrit", "py", "twitter", "cityfalconcom", "aapl", "ing", "ios", "samsung", "ipad", "phones", "cityfalconcom", "us", "bitly", "utmmpaign", "etexclusivecom", "cityfalcon", "owler", "com", "stock", "stocks", "buy", "bitly", "dlvrit", "alexa", "zprio", "billion", "seekalphacom", "et", "alphet", "seekalpha", "googl", "zprio", "trad", "jt", "windows", "adw", "ifttt", "ihadvfn", "nmona", "pphppid", "st", "bza", "twits", "biness", "tim", "ba", "se", "rat", "article"]


    #prep stocks_list_shortened
    stocks_list_shortened_dict  = update_shortened_company_file(df_stocks_list_file, corp_stopwords)
    stocks_list_shortened       = list(stocks_list_shortened_dict.values())
    
    #prep variables
    split_tweets = []
    output = []
    for tweet in input_list:
        split_tweet_pre = tweet.split(" ")
        split_tweet = []
        for word in split_tweet_pre:
            #split_tweet = split_tweet + [" " + word.lower() + " "]
            split_tweet = split_tweet + [word.lower()]
        split_tweets = split_tweets + [split_tweet]

    #removal of words
    for tweet in split_tweets:
        for word in reversed(tweet):
            Removed = False
            # remove words containing "x"
            for char in death_characters:
                if char in word:
                    tweet.remove(word)
                    Removed = True
                    break
            if Removed == False:
                for char in tickers_list + stopwords_english + corp_stopwords + misc_stopwords:
                    if char == word:
                        tweet.remove(word)
                        S = False
                        break
            # remove words equalling stop words
        
        
    #finalise and remove stock names
    output = []
    iteration_list = list(reversed(stocks_list)) + list(reversed(stocks_list_shortened))
    for split_tweet in split_tweets:
        #recombined_tweet = list(map(lambda x: x.strip(), split_tweet))
        recombined_tweet = " ".join(split_tweet)#.replace("  "," ")
        #recombined_tweet = " ".join(recombined_tweet)#.replace("  "," ")
        for stock_name in iteration_list:
            recombined_tweet = recombined_tweet.replace(stock_name, "")
        output = output + [recombined_tweet]
    
    return output




def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))


def return_subject_keys(df_prepped_tweets_company_agnostic, topic_qty = 10, enforced_topics_dict=None, stock_names_list=None, words_to_remove = None, 
                        return_LDA_model=True, return_png_visualisation=False, return_html_visualisation=False, 
                        alpha=0.1, apply_IDF=True, cores=2):
    output = []

    data = df_prepped_tweets_company_agnostic
    data_words = list(sent_to_words(data))
    if return_LDA_model < return_html_visualisation:
        raise ValueError("You must return the LDA visualisation if you return the LDA model")

       
    if return_png_visualisation==True:
        long_string = "start"
        for w in data_words:
            long_string = long_string + ',' + ','.join(w)
        wordcloud = WordCloud(background_color="white", max_words=1000, contour_width=3, contour_color='steelblue')
        wordcloud.generate(long_string)
        wordcloud.to_image()
        output = output + [wordcloud]
    else:
        output = output + [None]
    
    if return_LDA_model==True:
        # Create Dictionary
        id2word = corpora.Dictionary(data_words)

        # Create Corpus
        texts = data_words

        # Term Document Frequency
        corpus = [id2word.doc2bow(text) for text in texts]

        #translate the enforced_topics_dict input
        eta = None
        if enforced_topics_dict != None:
            eta = np.zeros(len(id2word))
            offset = 1
            for group_num in range(len(enforced_topics_dict)):
                for word in enforced_topics_dict[group_num]:
                    try: 
                        word_id = id2word.token2id[word]
                        eta[word_id] = group_num + offset
                    except:
                        a=1

        #apply IDF
        if apply_IDF == True:
            # create tfidf model
            tfidf = TfidfModel(corpus)

            # apply tfidf to corpus
            corpus = tfidf[corpus]
        
        
        
        # Build LDA model
        lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=topic_qty,
                                               eta=eta,
                                               alpha = alpha, # controls topic sparsity
                                               #beta = beta, # controls word sparsity
                                               workers=cores)
        
        # Print the Keyword in the 10 topics
        #pprint(lda_model.print_topics())
        doc_lda = lda_model[corpus]
        topic_model_dict = {"lda_model" : lda_model, "doc_lda" : doc_lda, "corpus" : corpus, "id2word" : id2word}
        output = output + [topic_model_dict]
    else:
        output = output + [None]
        
        
    if return_html_visualisation==True:
        pyLDAvis.enable_notebook()
        LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
        output = output + [LDAvis_prepared]
    else:
        output = output + [None]
    
    return tuple(output)



def load_topic_clusters(file_location_wordcloud, file_location_topic_model_dict, file_location_visualisation):

    #if file_location_wordcloud != None:
    #    with open(file_location_wordcloud, "rb") as file:
    #        wordcloud = pickle.load(file)

    if file_location_topic_model_dict != None:
        with open(file_location_topic_model_dict, "rb") as file:
            topic_model_dict = pickle.load(file)

    #if file_location_visualisation != None:
    #    with open(file_location_visualisation, "rb") as file:
    #        visualisation = pickle.load(file)

    return None, topic_model_dict, None


#def save_topic_clusters(output_folder_name, run_name, wordcloud=None, topic_model_dict=None, visualisation=None):
def save_topic_clusters(wordcloud=None, topic_model_dict=None, visualisation=None, file_location_wordcloud=None, file_location_topic_model_dict=None, file_location_visualisation=None):
    if wordcloud != None:
        wordcloud.to_file(file_location_wordcloud)
    
    #"lda_model" : lda_model, "doc_lda" : doc_lda, "corpus" : corpus, "id2word" : id2word}
    if topic_model_dict != None:
        file_path = file_location_topic_model_dict
        #LDAvis_prepared = gensimvis.prepare(topic_model_dict["doc_lda"], topic_model_dict["corpus"], topic_model_dict["id2word"])
            
        with open(file_path, "wb") as file:
            pickle.dump(topic_model_dict, file)
    
    if visualisation != None:
        pyLDAvis.save_html(visualisation, file_location_visualisation)
        
    
def generate_topic_model(raw_twitter_data, twitter_data_filters_dict, period_start, period_end, relavance_lifetime, tweet_ratio_removed, df_stocks_list_file, topic_qty, alpha, apply_IDF=True, enforced_topics_dict=None):
    print("-------------------------------- Importing Sentiment Data --------------------------------")
    print(datetime.now().strftime("%H:%M:%S"))
    df_prepped_tweets                           = import_twitter_data_period(raw_twitter_data, twitter_data_filters_dict, period_start, period_end, relavance_lifetime)
    print("-------------------------------- Prepping Sentiment Data --------------------------------")
    print(datetime.now().strftime("%H:%M:%S"))
    df_prepped_tweets_company_agnostic          = prep_twitter_text_for_subject_discovery(df_prepped_tweets["body"][::tweet_ratio_removed], df_stocks_list_file=df_stocks_list_file)
    print("-------------------------------- Creating Subject Keys --------------------------------")
    print(datetime.now().strftime("%H:%M:%S"))
    wordcloud, topic_model_dict, visualisation  = return_subject_keys(df_prepped_tweets_company_agnostic, topic_qty = topic_qty, alpha=alpha, apply_IDF=apply_IDF,
                                                                      enforced_topics_dict=enforced_topics_dict, return_LDA_model=True, return_png_visualisation=True, return_html_visualisation=True)
    return wordcloud, topic_model_dict, visualisation
    
    
    

def retrieve_or_generate_topic_model(raw_twitter_data, twitter_data_filters_dict, period_start, period_end, num_topics, alpha, tweet_ratio_removed, force_regeneration=False):
    global precalculated_assets_locations_dict
    file_string = str(period_start).replace(":","") + "_" + str(period_end).replace(":","") + "_" + str(num_topics) + "_" + str(alpha) + "_" + str(tweet_ratio_removed)
    folder_path = precalculated_assets_locations_dict["root"] + precalculated_assets_locations_dict["topic_models"]
    file_location_wordcloud         = folder_path + "wordcloud_" + file_string + ".png"
    file_location_topic_model_dict  = folder_path + "topic_model_dict_" + file_string + ".pkl"
    file_location_visualisation     = folder_path + "visualisation_" + file_string + '.html'
    
    if not os.path.exists(file_location_wordcloud) or not os.path.exists(file_location_topic_model_dict) or not os.path.exists(file_location_visualisation) or force_regeneration == True:
        wordcloud, topic_model_dict, visualisation = generate_topic_model(raw_twitter_data, twitter_data_filters_dict, period_start, period_end, relavance_lifetime, tweet_ratio_removed, df_stocks_list_file, topic_qty, alpha)
        save_topic_clusters(wordcloud, topic_model_dict, visualisation, file_location_wordcloud, file_location_topic_model_dict, file_location_visualisation)
    else:
        print("----------- Importing Topic Model From Previous Run ---------------")
        wordcloud, topic_model_dict, visualisation = load_topic_clusters(file_location_wordcloud, file_location_topic_model_dict, file_location_visualisation)
    return wordcloud, topic_model_dict, visualisation



def export_words_within_topics_count(lda_model):
    strings_dict = dict()
    for topic in lda_model.print_topics():
        tuples = topic[1].split("+")
        for tup in tuples:
            word = tup.split("+")[0].split("*")[1]
            word = re.sub('[^a-zA-Z]', '', word)
            num  = tup.split("+")[0].split("*")[0]
            if not word in strings_dict.keys():
                strings_dict[word] = 0
            strings_dict[word] += float(num)

    words_within_topics_count = pd.DataFrame(list(strings_dict.items()), columns=['keys', 'values'])
    words_within_topics_count.sort_values(by="values", ascending=False, inplace=True)
    model_time = datetime.now()
    model_time = model_time.strftime("%Y%m%d_%H%M")
    words_within_topics_count.to_csv("C:\\Users\\Fabio\\OneDrive\\Documents\\Studies\\Final Project\\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\\test books\\output\\word_topics" + model_time + ".csv")


#parameters
tweet_ratio_removed = int(1e4)
topic_qty = 7
alpha=0.1


raw_twitter_data = r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\twitter data\Tweets about the Top Companies from 2015 to 2020\Tweet.csv\Tweet.csv"
period_start = datetime.strptime('01/05/17 00:00:00', '%d/%m/%y %H:%M:%S')
period_end = datetime.strptime('15/05/18 00:00:00', '%d/%m/%y %H:%M:%S')
relavance_lifetime      = 2*24*60*60
relavance_lifetime      = 2*24*60*60
precalculated_assets_locations_dict = {
    "root" : "C:\\Users\\Fabio\\OneDrive\\Documents\\Studies\\Final Project\\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\\precalculated_assets\\",
    "topic_models" : "topic_models\\"
}
presaved_naming_convention_topic_models = ["period_start" , "period_end" , "seconds_per_timestep" , "num_topics" , "relative_halflife" , "relative_lifetime", "tweet_ratio_removed"]



enforced_topics_dict = [
    ['investment', 'financing', 'losses'],
    ['risk', 'exposure', 'liability'],
    ["financial forces" , "growth", "interest rates"]]
df_stocks_list_file     = pd.read_csv(r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\data\stock_info.csv")
output_folder_name = "C:\\Users\\Fabio\\OneDrive\\Documents\\Studies\\Final Project\\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\\test books\\output\\"



wordcloud, topic_model_dict, visualisation = retrieve_or_generate_topic_model(raw_twitter_data, None, period_start, period_end, topic_qty, alpha, tweet_ratio_removed)

##data prep
##print(datetime.now().strftime("%H:%M:%S"))
#df_prepped_tweets                           = import_twitter_data_period(input_file, dict(), period_start, period_end, relavance_lifetime)
##print("--------------------------------1--------------------------------")
##print(datetime.now().strftime("%H:%M:%S"))
#df_prepped_tweets_company_agnostic          = prep_twitter_text_for_subject_discovery(df_prepped_tweets["body"][::tweet_ratio_removed], df_stocks_list_file=df_stocks_list_file)
##print("--------------------------------2--------------------------------")
##print(datetime.now().strftime("%H:%M:%S"))
#wordcloud, topic_model_dict, visualisation  = return_subject_keys(df_prepped_tweets_company_agnostic, topic_qty = topic_qty, alpha=alpha, apply_IDF=True,
#                                                                                     enforced_topics_dict=enforced_topics_dict, return_LDA_model=True, return_png_visualisation=True, return_html_visualisation=True)
##print("--------------------------------3--------------------------------")
##print(datetime.now().strftime("%H:%M:%S"))
#save_load_subject_keys(output_folder_name, str(tweet_ratio_removed), wordcloud=wordcloud, topic_model_dict=topic_model_dict, visualisation=visualisation)
#export_words_within_topics_count(topic_model_dict["lda_model"])
#print("--------------------------------4--------------------------------")
#print(datetime.now().strftime("%H:%M:%S"))

----------- Importing Topic Model From Previous Run ---------------


UnpicklingError: invalid load key, '\x0d'.

In [ ]:
topic_model_dict

{'lda_model': <gensim.models.ldamulticore.LdaMulticore at 0x2b5b0467c90>,
 'doc_lda': <gensim.interfaces.TransformedCorpus at 0x2b5af47e4d0>,
 'corpus': <gensim.interfaces.TransformedCorpus at 0x2b5be821b10>,
 'id2word': <gensim.corpora.dictionary.Dictionary at 0x2b606176150>}

In [5]:


def return_terms_for_company(company_ticker, df_stocks_list_file, stocks_list_shortened_dict):
    
    company_longform_name   = df_stocks_list_file["Name"][df_stocks_list_file["Ticker"] == company_ticker.upper()].values
    if len(company_longform_name) > 1:
        raise ValueError("ERROR: More then one company name found for ticker: " + company_ticker)
    if len(company_longform_name) == 0:
        raise ValueError("ERROR: No company name found for ticker: " + company_ticker)
    company_longform_name   = company_longform_name[0]
    
    if company_longform_name in stocks_list_shortened_dict.keys():
        return [company_ticker] + [company_longform_name] + [stocks_list_shortened_dict[company_longform_name]]
        
    else:
        return [company_ticker] + [company_longform_name]


def filter_tweets_for_company(df_prepped_tweets, terms_for_company_list, length_test):
    df_prepped_tweets = df_prepped_tweets[::length_test]
    index_set = set()
    
    pattern = re.compile('|'.join(terms_for_company_list))
    
    # filter for rows containing any of the substrings in the 'column_name' column
    filtered_df = df_prepped_tweets[df_prepped_tweets['body'].apply(lambda x: bool(pattern.search(x)))]
    
    return filtered_df


def update_shortened_company_file(df_stocks_list_file, corp_stopwords, file_location=None):
    stocks_list         = list(df_stocks_list_file["Name"].map(lambda x: x.lower()).values)
            
    stocks_list_shortened_dict = dict()
    for stock_name in stocks_list:
        shortened = False
        stock_name_split = stock_name.split(" ")
        for word in reversed(stock_name_split):
            for stop in corp_stopwords:
                if stop == word:
                    stock_name_split.remove(word)
                    shortened = True
        if shortened == True:
            stocks_list_shortened_dict[stock_name] = " ".join(stock_name_split)
    
    return stocks_list_shortened_dict

def return_topic_weight(text_body, id2word, lda_model):
    bow_doc = id2word.doc2bow(text_body.split(" "))
    doc_topics = lda_model.get_document_topics(bow_doc)
    return doc_topics


def annotate_tweets_senti_scores_and_topics(df_tweets_company, sentiment_method, topic_model_dict):
    
    columns_to_add = ["~sent_overall"]
    for num in range(topic_model_dict["lda_model"].num_topics):
        columns_to_add = columns_to_add + ["~sent_topic_W" + str(num)]
    
    df_tweets_company[columns_to_add] = float("nan")
    count = 0 # FG_Counter
    
    for tweet_id in df_tweets_company.index:
        
        text = df_tweets_company["body"][tweet_id]
        sentiment_value = sentiment_method.polarity_scores(text)["compound"]
        topic_tuples = return_topic_weight(text, topic_model_dict["id2word"], topic_model_dict["lda_model"])
        if len(topic_tuples) == topic_model_dict["lda_model"].num_topics:
            topic_weights = [t[1] for t in topic_tuples]
        else:
            topic_weights = list(np.zeros(topic_model_dict["lda_model"].num_topics))
            for tup in topic_tuples:
                topic_weights[tup[0]] = tup[1]
        sentiment_analysis = [sentiment_value] + topic_weights
        df_tweets_company.loc[tweet_id, columns_to_add] = sentiment_analysis
    return df_tweets_company

def generate_datetimes(period_start, period_end, seconds_per_time_steps):
    format_str = '%Y%m%d_%H%M%S'  # specify the desired format for the datetime strings
    current_time = period_start
    datetimes = []
    while current_time <= period_end:
        datetimes.append(current_time.strftime(format_str))
        current_time += timedelta(seconds=seconds_per_time_steps)
    return datetimes

# Example usage:

#def return_sentiment_score():
    
def update_tweet_cohort(df_tweets, cohort_start_secs, cohort_end_secs):
    epoch_time          = datetime(1970, 1, 1)
    df_tweets        = df_tweets[df_tweets["post_date"] <= cohort_end_secs]
    df_tweets        = df_tweets[df_tweets["post_date"] >= cohort_start_secs]
    return df_tweets


def return_sentiment_company(df_tweets_company, period_start, period_end, seconds_per_time_steps, num_topics, relavance_halflife, relavance_lifetime, weighted_topic_relavance = False):
    index = generate_datetimes(period_start, period_end, seconds_per_time_steps)
    #df_sentiment_scores = pd.DataFrame()
    columns = []
    for i in range(num_topics):
        columns = columns + ["~senti_score_t" + str(i)]
    df_sentiment_scores = pd.DataFrame(index=index, columns=columns)
    
    #create the initial cohort of tweets to be looked at in a time window
    epoch_time          = datetime(1970, 1, 1)
    tweet_cohort_start  = (period_start - epoch_time) - timedelta(seconds=relavance_lifetime)
    tweet_cohort_end    = (period_start - epoch_time)
    tweet_cohort_start  = tweet_cohort_start.total_seconds()
    tweet_cohort_end    = tweet_cohort_end.total_seconds()
    tweet_cohort        = update_tweet_cohort(df_tweets_company, tweet_cohort_start, tweet_cohort_end)
    
    for time_step in index:
        senti_scores = list(np.zeros(num_topics))
        pre_calc_time_overall = np.exp((- 3 / relavance_lifetime) * (tweet_cohort.loc[:, "post_date"] - tweet_cohort_start)) * tweet_cohort.loc[:, "~sent_overall"]
        for topic_num in range(num_topics):
            score_numer = sum(pre_calc_time_overall * tweet_cohort.loc[:, "~sent_topic_W" + str(topic_num)])
            score_denom = sum(np.exp((- 3 / relavance_lifetime) * (tweet_cohort.loc[:, "post_date"] - tweet_cohort_start)) * tweet_cohort.loc[:, "~sent_topic_W" + str(topic_num)])
            if score_numer > 0 and score_denom > 0:
                senti_scores[topic_num] = score_numer / score_denom
        #update table
        df_sentiment_scores.loc[time_step, :] = senti_scores
        #update tweet cohort
        tweet_cohort_start += seconds_per_time_steps
        tweet_cohort_end   += seconds_per_time_steps
        tweet_cohort        = update_tweet_cohort(df_tweets_company, tweet_cohort_start, tweet_cohort_end)
    
    return df_sentiment_scores


    
def return_topic_company_sentiment(length_test, df_prepped_tweets, sentiment_model, topic_model_dict, pred_output_and_tickers_combos_list, period_start, period_end, seconds_per_time_steps, relavance_halflife, relavance_lifetime, weighted_topic_relavance = False):
    #parameters
    corp_stopwords      = [".com", "company", "corp", "froup", "fund", "gmbh", "global", "incorporated", "inc.", "inc", "tech", "technology", "technologies", "trust", "limited", "lmt", "ltd"]
    df_sentiment_data_prepped = pd.DataFrame()
    
    #calc shortened names
    stocks_list_shortened_dict = update_shortened_company_file(df_stocks_list_file, corp_stopwords)
    
    #list unique companies
    tickers_list = set()
    for output_and_tickers in pred_output_and_tickers_combos_list:
        tickers_list.add(output_and_tickers[0])
    tickers_list = list(tickers_list)
        
    for ticker in tickers_list[0]:
        terms_for_company_list      = return_terms_for_company(ticker, df_stocks_list_file, stocks_list_shortened_dict)
        df_tweets_company           = filter_tweets_for_company(df_prepped_tweets, terms_for_company_list, length_test)
        df_tweets_company_senti     = annotate_tweets_senti_scores_and_topics(df_tweets_company, sentiment_model, topic_model_dict)
        df_sentiment_company_single = return_sentiment_company(df_tweets_company_senti, period_start, period_end, seconds_per_time_steps, topic_model_dict["lda_model"].num_topics, relavance_halflife, relavance_lifetime, weighted_topic_relavance = weighted_topic_relavance)
        #df_sentiment_data_prepped  = pd.concat([df_sentiment_data_prepped, df_sentiment_company_single])
    
    df_sentiment_data_prepped = df_sentiment_company_single
    return df_sentiment_data_prepped

df_prepped_tweets   = df_prepped_tweets
topic_model_dict    = topic_model_dict

length_test             = int(1e2)
pred_output_and_tickers_combos_list = [("aapl", "<CLOSE>"), ("axp", "<HIGH>")]
df_prepped_tweets       = df_prepped_tweets
#FG action the format of the datetimes needs to be aligned
period_start            = datetime(2017, 5, 1, 0, 0, 0)
period_end              = datetime(2017, 5, 15, 0, 0, 0)
seconds_per_time_steps  = 60*60  # time step length in seconds
relavance_halflife      = 4*60*60
relavance_lifetime      = 2*24*60*60





#start_time  = datetime(2016, 5, 1, 0, 0, 0)
#end_time    = datetime(2017, 5, 1, 0, 0, 0)
#time_step   = 60*60  # time step length in seconds




#sentiment_model = SentimentIntensityAnalyzer()
df_sentiment_data_prepped = return_topic_company_sentiment(length_test, df_prepped_tweets, SentimentIntensityAnalyzer(), topic_model_dict, pred_output_and_tickers_combos_list, period_start, period_end, seconds_per_time_steps, relavance_halflife, relavance_lifetime, weighted_topic_relavance = False)

In [ ]:
print("hello")

hello
